In [ ]:
# Import necessary libraries and functions
import os
import shutil
import warnings
from time import time
import zipfile

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
from scipy.stats import norm

from sklearn.model_selection import (StratifiedKFold, cross_val_score, GridSearchCV,
                                     train_test_split)
from sklearn.metrics import (balanced_accuracy_score, classification_report,
                             matthews_corrcoef, confusion_matrix, mean_squared_error,
                             roc_curve, auc, roc_auc_score, accuracy_score,
                             f1_score, precision_score)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier

from mlxtend.plotting import plot_decision_regions


# Copy Datasets
source_file_path = '/input/'
destination_file_path = '/content/data.zip'
shutil.copy(source_file_path, destination_file_path)

# Uzip the zipped dataset file
zip_file_path = '/content/data.zip'
extract_to_path = '/content/data/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

In [ ]:
# Suppress all warnings
warnings.filterwarnings('ignore')

############################################################################################# Plots
# Function to plot histograms for different performance metrics
def plot_histograms(auc_list, test_acc_list, mcc_list, bal_acc_list, model_name, method_name):
    metrics = [auc_list, test_acc_list, mcc_list, bal_acc_list]
    metric_names = ["AUC", "Test Accuracy", "MCC", "Balanced Accuracy"]

    plt.figure(figsize=(15, 10))

    # iterate over the metrics and plot histogram
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i + 1)
        plt.hist(metric, bins=20, edgecolor='black', alpha=0.7)
        plt.title(metric_names[i])
        plt.xlabel('Value')
        plt.ylabel('Frequency')

    plt.tight_layout()

    # filename and saving the figure
    directory = "/content/report/plots"
    filename = os.path.join(directory, f"{model_name}_{method_name}.png")

    plt.savefig(filename, dpi=300)
    plt.close()

# Function to plot AUC with error bars for different models and methods
def plot_auc_with_error_bars(results_df):
    # Extracting relevant data for plotting
    labels = results_df["Model"] + "-" + results_df["Feature Reduction Method"]
    auc_means = results_df["AUC Mean"].values
    auc_stds = results_df["AUC Std"].values

    # Setting the figure size and creating the bar plot with error bars
    plt.figure(figsize=(14, 8))
    plt.bar(range(len(labels)), auc_means, yerr=auc_stds, align='center', alpha=0.7, ecolor='black', capsize=10)
    plt.ylabel('AUC')
    plt.xticks(range(len(labels)), labels, rotation=45, ha="right")

    # Annotations for better clarity
    for i, v in enumerate(auc_means):
        plt.text(i, v + 0.01, f"{v:.2f}", ha='center', va='bottom', fontsize=8, color='black')

    plt.tight_layout()
    plt.title('AUC with Error Bars for Different ML Model-FR Method Combinations', y=1.05)

    # Saving the plot
    plt.savefig("/content/report/auc_barplot_with_error_bars.png", bbox_inches='tight')
    plt.close()

def plot_performance_heatmap(df, metric, save_path):

  df[['Mean', 'CI']] = df[metric].str.split('±', expand=True)
  df['Mean'] = df['Mean'].astype(float)

  # Creating a pivot table for the heatmap
  pivot_table = df.pivot("Model", "Feature Reduction Method", "Mean")

  # Plotting the heatmap
  plt.figure(figsize=(10, 8))
  ax = sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap="YlGnBu")

  # Annotation settings
  fontsize_mean = 10 
  fontsize_ci = 8
  mean_color = 'white'
  ci_color = 'darkorange'

  # Adding the confidence interval and mean to the heatmap cells
  for i, row in enumerate(pivot_table.values):
      for j, _ in enumerate(row):
          mean_value = pivot_table.iloc[i, j]
          mean_text = f"{mean_value:.4f}"
          ci_text = df.loc[(df['Model'] == pivot_table.index[i]) & (df['Feature Reduction Method'] == pivot_table.columns[j]), 'CI'].values[0]
          
          ax.text(j + 0.5, i + 0.7, f'±{ci_text}', horizontalalignment='center', verticalalignment='center', color=ci_color, fontsize=fontsize_ci)

  plt.title(f'Heatmap of {metric} across Models and FR Methods')
  plt.tight_layout()

  # Save the heatmap
  plt.savefig(save_path)
  plt.close()

# zip the contents of a folder
def zip_folder(folder_path, output_filename):
    shutil.make_archive(output_filename, 'zip', folder_path)

    return f"{output_filename}.zip"

# Configure pandas to display all rows in google colab
pd.set_option('display.max_rows', None)
# Ensure the directory for plots exists, create if it does not
os.makedirs("/content/report/plots", exist_ok=True)

########################################################################################### Load dataset
# Load dataset
def load_dataset(sheet_name, remove_unknowns=True):
    path = '/content/data/merge_m1_esi_plus_minus_norm.xlsx'
    dataset = pd.read_excel(path, sheet_name=sheet_name, header=0)
    dataset = dataset.sample(frac=1, random_state=666).reset_index(drop=True)
    dataset = dataset.drop(columns=['eye', 'gender', 'age'])
    # Optionally remove columns that start with 'unknown'
    if remove_unknowns:
        dataset = dataset.loc[:, ~dataset.columns.str.startswith('unknown')]
    return dataset

# Prepare the data for training
def prepare_data(dataset):
    # Extract labels for training
    train_label = dataset['class']
    # Drop the first and last columns from the dataset for training data
    train_data = dataset.drop(columns=[dataset.columns[0], dataset.columns[-1]])
    return train_data, train_label, dataset.columns.tolist()

# Get the dataset and preprocess it based on feature selection
def get_data(sheet_name, train_features):
    if train_features == 'Only known metabolites':
        dataset = load_dataset(sheet_name, remove_unknowns=True)
    else:
        dataset = load_dataset(sheet_name, remove_unknowns=False)
    return prepare_data(dataset)

# Select the data and feature types for the experiment
data = "M1 Compounds ESI+ and ESI- T." # @param ["M1 Compounds ESI+ T.", "M1 Compounds ESI- T.", "M4 Compounds ESI+ T.", "M4 Compounds ESI- T.", "M1 Compounds ESI+ and ESI- T."]
train_features = "All metabolites"

# Retrieve the prepared data for training
train_data, train_label, columns_list = get_data(data, train_features)

# Convert the training data and labels to NumPy arrays of type 'float32'
train_data = np.array(train_data).astype('float32')
train_label = np.array(train_label).astype('float32')

# Assign feature reduction method parameter
fr_method = "All methods"

# Assign machine learning model parameter
ml_model = "All models"

models_dict = {
    "XGBoost": lambda: xgb.XGBClassifier(objective="binary:logistic"),
    "Random Forest": lambda: RandomForestClassifier(),
    "Support Vector Machine": lambda: SVC(probability=True),
    "MLP": lambda: MLPClassifier(),
    "SGD": lambda: SGDClassifier(loss='log'),
    "Logistic Regression": lambda: LogisticRegression(),
    "k-NN": lambda: KNeighborsClassifier(n_neighbors=5),
    "Dummy Classifier": lambda: DummyClassifier(strategy='most_frequent')
}

param_grid_search = {
    "XGBoost": {
        'max_depth': [3, 4],  # default 3
        'learning_rate': [0.1, 0.01],  # default 0.1
        'n_estimators': [100, 150],  # default 100
        'min_child_weight': [1, 2],  # default 1
        'gamma': [0, 0.1],  # default 0
        'subsample': [1.0],  # default 1
        'colsample_bytree': [1.0],  # default 1
    },

    "Random Forest": {
        'n_estimators': [100, 200],  # Default is 100
        'criterion': ['gini', 'entropy'],  # Default is 'gini'
        'max_depth': [None, 10],  # Default is None (full depth)
        'min_samples_split': [2, 4],  # Default is 2
        'min_samples_leaf': [1],  # Default is 1
        'max_features': ['auto'],  # Default is 'auto' (sqrt)
        'bootstrap': [True]  # Default is True
    },
    "Support Vector Machine": {
        'C': [0.1, 1, 10],  # Default is 1.0
        'gamma': ['scale', 'auto'],  # Default is 'scale'
        'kernel': ['rbf']  # Fixed as rbf for this search
    },
    "MLP": {
        'hidden_layer_sizes': [(100,), (50,50)],  # Default is (100,)
        'activation': ['logistic', 'relu'],  # Default is 'relu'
        'solver': ['adam', 'sgd'],  # Default is 'adam'
        'alpha': [0.0001, 0.001],  # Default is 0.0001
        'learning_rate': ['constant', 'adaptive'],  # Default is 'constant'
        'max_iter': [200],  # Default is 200
    },
    "SGD": {
        'loss': ['log'],  # default 'hinge'
        'penalty': ['l2', 'l1', 'elasticnet'],  # default 'l2'
        'alpha': [0.0001, 0.001],  # default 0.0001
        'learning_rate': ['optimal', 'adaptive'],  # default 'optimal'
    },
    "Logistic Regression": {
        'penalty': ['l2'],  # default penalty
        'C': [1.0, 0.1],  # default value is 1.0
        'solver': ['lbfgs', 'newton-cg'],  # default solver is 'lbfgs'
        'max_iter': [100],  # default is 100
        'class_weight': [None, 'balanced']  # default is None
    },
    "k-NN": {
        'n_neighbors': [3, 5],  # default is 5
        'weights': ['uniform', 'distance'],  # default is 'uniform'
        'algorithm': ['auto', 'ball_tree'],  # default is 'auto'
        'leaf_size': [20, 30],  # default is 30
        'p': [2]  # default is 2 (Euclidean distance)
    },
    "Dummy Classifier": {
        "strategy": ["most_frequent"]
    }
}


# Initialize results DataFrame
results_df = pd.DataFrame(columns=["Model", "AUC", "AUC SD", "MCC", "MCC SD","Precision", "Precision SD", "Sensitivity", "Sensitivity SD", "Specificity", "Specificity SD", "F1-score", "F1-score SD", "Balanced Accuracy", "Balanced Accuracy SD", "Test Accuracy", "Test Accuracy SD", "Train and test time"])
auc_raw_df = pd.DataFrame(columns=["Model","AUC Scores"])

# Training and evaluating models function
def train_and_evaluate(model_factory, x_data, y_data, model_name):

    t0 = time()
    # Lists to store metrics for each iteration
    mcc_list, bal_acc_list, cv_acc_list, specificity_list, sensitivity_list, precision_list, f1_score_list, test_acc_list, roc_auc_list = [], [], [], [], [], [], [], [], []
    auc_sc = None
    model = model_factory()

    # Set up stratified
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    # Conduct the k-fold cross-validation
    for train_index, test_index in skf.split(x_data, y_data):
        # Split the data into training and testing sets for the current fold
        x_train_fold, x_test_fold = x_data[train_index], x_data[test_index]
        y_train_fold, y_test_fold = y_data[train_index], y_data[test_index]

        grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid_search[model_name],
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=777),
                           scoring='balanced_accuracy',
                           verbose=1)

        grid_search.fit(x_train_fold, y_train_fold)
        new_model = grid_search.best_estimator_

        # Predict on the test fold and calculate accuracy
        if model_name == 'Support Vector Machine':
          new_model.probability = True
        y_pred = new_model.predict(x_test_fold)
        test_acc_list.append(accuracy_score(y_test_fold, y_pred))

        # Predict probabilities for the AUC calculation
        y_score = new_model.predict_proba(x_test_fold)[:, 1]
        fpr, tpr, _ = roc_curve(y_test_fold, y_score)

        # Calculate and store AUC, MCC, and balanced accuracy
        roc_auc_list.append(auc(fpr, tpr))
        mcc_list.append(matthews_corrcoef(y_test_fold, y_pred))
        bal_acc_list.append(balanced_accuracy_score(y_test_fold, y_pred))

        # Calculate precision, F1-score, specificity, and sensitivity
        precision = precision_score(y_test_fold, y_pred)
        f1_score_com = f1_score(y_test_fold, y_pred)
        precision_list.append(precision)
        f1_score_list.append(f1_score_com)

        tn, fp, fn, tp = confusion_matrix(y_test_fold, y_pred).ravel()
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)

        specificity_list.append(specificity)
        sensitivity_list.append(sensitivity)

    # Compile all the collected metrics into a summary dictionary
    auc_sc = {
        "Model": model_name,
        "AUC Scores":roc_auc_list
          }
    results = {
        "Model": model_name,
        "AUC": round(np.mean(roc_auc_list),4),
        "AUC SD": round(np.std(roc_auc_list), 4),
        "MCC": round(np.mean(mcc_list),4),
        "MCC SD": round(np.std(mcc_list), 4),
        "Precision": round(np.mean(precision_list),4),
        "Precision SD": round(np.std(precision_list), 4),
        "Sensitivity": round(np.mean(sensitivity_list),4),
        "Sensitivity SD": round(np.std(sensitivity_list), 4),
        "Specificity": round(np.mean(specificity_list),4),
        "Specificity SD": round(np.std(specificity_list), 4),
        "F1-score": round(np.mean(f1_score_list),4),
        "F1-score SD": round(np.std(f1_score_list), 4),
        "Balanced Accuracy": round(np.mean(bal_acc_list),4),
        "Balanced Accuracy SD": round(np.std(bal_acc_list), 4),
        "Test Accuracy": round(np.mean(test_acc_list),4),
        "Test Accuracy SD": round(np.std(test_acc_list), 4),
        "Train and test time": round(time()-t0, 2)
    }
    return results, auc_sc

# Execute training and evaluation
if ml_model == "All models":
    # Loop through each model and method to train and evaluate
    for model_name, model in tqdm(models_dict.items(), desc="Status"):
        # Train the model and collect results
        result, auc_scroe = train_and_evaluate(model, train_data, train_label, model_name)
        results_df = results_df.append(result, ignore_index=True)
        auc_raw_df = auc_raw_df.append(auc_scroe, ignore_index=True)

# Save the aggregated results to an Excel file for further analysis
results_df.to_excel("/content/report/results.xlsx", index=False)
auc_raw_df.to_excel('/content/report/auc_scores_results.xlsx', index=False)

# Visualize the performance of models using AUC with error bars
#plot_auc_with_error_bars(results_df)

# Plot and save the heatmap for AUC and Balanced acuuracy
#plot_performance_heatmap(results_df, 'AUC', '/content/report/AUC_performance_heatmap.png')
#plot_performance_heatmap(results_df, 'Balanced Accuracy', '/content/report/Balanced_Accuracy_performance_heatmap.png')
#plot_performance_heatmap(results_df, 'MCC', '/content/report/MCC_performance_heatmap.png')

# Package all results and plots into a downloadable zip file
zip_path = zip_folder("/content/report", "/content/report")
# Print out the path to the zipped results
print(f"The results of this experiment, as well as the plots, are available for download in a zipped folder at the address: {zip_path}")